In [6]:
import time
import random
from collections import deque

# Ucitavanje grafa

In [7]:
def load_graph_from_mtx(filename):
    graph = {}
    with open(filename, 'r') as file:
        for i, line in enumerate(file):
            # preskocimo prva tri reda (jer su tu podaci o broju grana, cvorova...)
            if i < 3:
                continue
            
            nodes = line.strip().split()
            node1, node2 = int(nodes[0]), int(nodes[1])

            if node1 not in graph:
                graph[node1] = set()
            if node2 not in graph:
                graph[node2] = set()

            graph[node1].add(node2)
            graph[node2].add(node1)

    return graph


# Funkcije za racunanje betweenness centralnosti

In [8]:
def bfs_shortest_paths(graph, start):
    dist = {node: float('inf') for node in graph} 
    dist[start] = 0
    queue = deque([start])
    paths = {node: [] for node in graph}
    paths[start] = [start]
    
    while queue:
        current = queue.popleft()
        
        for neighbor in graph[current]:
            if dist[neighbor] == float('inf'):
                dist[neighbor] = dist[current] + 1
                queue.append(neighbor)
            
            if dist[neighbor] == dist[current] + 1:
                paths[neighbor].append(current)
    
    return dist, paths

def calculate_betweenness(graph):
    betweenness = {node: 0 for node in graph}
    
    for start in graph:
        dist, paths = bfs_shortest_paths(graph, start)
        
        for node in graph:
            if node == start:
                continue
            
            for neighbor in paths[node]:
                if neighbor != start:
                    betweenness[neighbor] += 1
    
    return betweenness

def betweenness_centralnost(node, betweenness):
    return betweenness.get(node, 0) 

# Pohlepni algoritam

In [9]:
def greedy_burning(graph):
    
    S=[] # sekvenca zapaljenih cvoreva
    B= set() # skup zapaljenih cvoreva
    NB= set(graph.keys()) # skup nezapaljenih cvoreva, inicajlno su to svi cvorevi grafa
    
    # ako imamo vise kandidata, biramo onog sa najvisim betweenness
    
    max_degree = max(len(graph[node]) for node in graph)
    candidates = [node for node in graph if len(graph[node]) == max_degree]
    current_node = None
    
    if len(candidates) > 1:
        betweenness = calculate_betweenness(graph)
        current_node = max(candidates, key=lambda node: betweenness_centralnost(node, betweenness))
    else:
        current_node = candidates[0]
   
    S.append(current_node)
    B.add(current_node)
    NB.remove(current_node)
    
    while len(B)<len(graph):
       
        # korak 2: zapaliti sve susjede (koji nisu zapaljeni) od cvorova u skupu zapaljenih, dodajemo ih u B, brisemo iz NB
        
        # Provjera: ako su svi cvorovi u NB zapravo cvorovi koji ce biti zapaljeni u narednom koraku (jer su direktni susjedi
        # zapaljenom cvoru nekom), onda cemo izabrati neki random cvor iz NB cisto kao simbol jos jednog koraka
        
        FB = []
        for b in B:
            neighbors_b1 = graph[b]
            for neighbor in neighbors_b1:
                if neighbor in NB:
                    if neighbor not in FB:
                        FB.append(neighbor)
             
        if set(FB) == NB:
            random_node = random.choice(list(NB))
            S.append(random_node)
          
        newlyburned = []
        for node in B:
            neighbors = graph[node]  # svi susjedi trenutnog zapaljenog cvora
            for neighbor in neighbors:
                if neighbor in NB:  # ako susjed nije zapaljen
                    newlyburned.append(neighbor)  # dodajemo ga u zapaljene
                    NB.remove(neighbor)  
        
        for new in newlyburned:
            B.add(new) #dodajemo sve susjede u zapaljene
        
        # pravimo listu onih cvorova za koje znamo da ce se u iducem koraku zapaliti. Bolje da izaberemo sljedeci direktni
        # cvor za koji znamo da nece biti zapaljen u narednom koraku
        
        FB2 = []
        for b in B:
            neighbors_b = graph[b]
            for neighbor in neighbors_b:
                if neighbor in NB:
                    if neighbor not in FB2:
                        FB2.append(neighbor)
       
        # korak 3: pronaci iduci cvor koji ce biti direktno zapaljen, dodati ga u S, B i izbrisati iz NB
        
        max_burned = -1
        best_node = None
        candidates_for_best = []
        
        if NB:
            for node in NB:
                if node not in FB2: 
                    notburned_neighbors = len([n for n in graph[node] if n not in B and n not in FB2])
                    if notburned_neighbors>max_burned:
                        max_burned = notburned_neighbors
                        
            for node in NB:
                if node not in FB2:
                    notburned_neighbors = len([n for n in graph[node] if n not in B and n not in FB2])
                    if notburned_neighbors == max_burned:
                        candidates_for_best.append(node) # u slucaju da imamo vise kandidata za najbolji cvor
                        
            
            
            if len(candidates_for_best)>1:# izbor na osnovu betweenness centralnosti
                 best_node = max(candidates_for_best, key=lambda node: betweenness_centralnost(node, calculate_betweenness(graph)))
            elif len(candidates_for_best)==1:
                best_node = candidates_for_best[0]
                
            if best_node is not None:
                S.append(best_node)
                B.add(best_node)
                NB.remove(best_node)
            else:
             # Ako nema nijednog najboljeg cvora znaci da su svi preostali cvorovi povezani i mogu biti zapaljeni
             # Dodajemo slucajni cvor iz preostalih nezapaljenih čvorova
                random_node = random.choice(list(NB))
                S.append(random_node)
                B.add(random_node)
                NB.remove(random_node)
           
            
        else:
            break
           
    return S
        
        

In [10]:
startTime = time.time()
filename= "grafovi/primjer.mtx"  
#filename = "grafovi/dolphins.mtx"

graph = load_graph_from_mtx(filename)
burn_order = greedy_burning(graph)

print("Redoslijed paljenja cvorova:", burn_order)
print("b(G)=",len(burn_order))
print("--- %s sekundi ---" % (time.time() - startTime))


Redoslijed paljenja cvorova: [1, 15, 17, 12]
b(G)= 4
--- 0.006853342056274414 sekundi ---
